In [ ]:
##Crew Standby Analytics


import pandas as pd
import datetime as dt
import copy
import numpy as np
from sklearn import linear_model
import sklearn.metrics as sm
from sklearn.linear_model import ElasticNet
import math 

from sklearn import ensemble 
from sklearn import datasets
from sklearn.utils import shuffle
from sklearn.metrics import mean_squared_error
import sklearn.neural_network as nn
import sklearn.preprocessing as sp
from sklearn.metrics import r2_score
from sklearn import preprocessing
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg
import random
import configparser



random.seed(10)
# get input data 

# config = configparser.ConfigParser()
# config.read('Configurations.ini')
#sDataFolder=config['INPUT FILES']['strPath']
#sDataFileName=config['INPUT FILES']['strFilename']

dfData=pd.read_csv('dfHistoricalData.csv',encoding='cp1252')

#data cleansing and imputing
dfData=dfData.dropna(axis=1,how='all')
dfData=dfData.fillna(0)    

dfData['SS22']=dfData['SS22_CS']+dfData['SS22_FS']+dfData['SS22_FSS']+dfData['SS22_IFS']+dfData['SS22_LS']+dfData['SS22_LSS']
dfData['SSS3']=dfData['SSS3_CS']+dfData['SSS3_FS']+dfData['SSS3_FSS']+dfData['SSS3_IFS']+dfData['SSS3_LS']+dfData['SSS3_LSS']
dfData['SSS1']=dfData['SSS1_CS']+dfData['SSS1_FS']+dfData['SSS1_FSS']+dfData['SSS1_IFS']+dfData['SSS1_LS']+dfData['SSS1_LSS']
dfData['SSS2']=dfData['SSS2_CS']+dfData['SSS2_FS']+dfData['SSS2_FSS']+dfData['SSS2_IFS']+dfData['SSS2_LS']+dfData['SSS2_LSS']
dfData['SSS0']=dfData['SSS0_CS']+dfData['SSS0_FS']+dfData['SSS0_FSS']+dfData['SSS0_IFS']+dfData['SSS0_LS']+dfData['SSS0_LSS']
dfData['SS60']=dfData['SS60_CS']+dfData['SS60_FS']+dfData['SS60_FSS']+dfData['SS60_IFS']+dfData['SS60_LS']+dfData['SS60_LSS']
dfData['SS80']=dfData['SS80_CS']+dfData['SS80_FS']+dfData['SS80_FSS']+dfData['SS80_IFS']+dfData['SS80_LS']+dfData['SS80_LSS']
dfData['SS77']=dfData['SS77_CS']+dfData['SS77_FS']+dfData['SS77_FSS']+dfData['SS77_IFS']+dfData['SS77_LS']+dfData['SS77_LSS']
dfData['SS33_55_88']=dfData['SS33_CS']+dfData['SS33_FS']+dfData['SS33_FSS']+dfData['SS33_IFS']+dfData['SS33_LS']+dfData['SS33_LSS']+dfData['SS55_CS']+dfData['SS55_FS']+dfData['SS55_FSS']+dfData['SS55_IFS']+dfData['SS55_LS']+dfData['SS55_LSS']+dfData['SS88_CS']+dfData['SS88_FS']+dfData['SS88_FSS']+dfData['SS88_IFS']+dfData['SS88_LS']+dfData['SS88_LSS']
dfData['SS33']=dfData['SS33_CS']+dfData['SS33_FS']+dfData['SS33_FSS']+dfData['SS33_IFS']+dfData['SS33_LS']+dfData['SS33_LSS']
dfData['SS55']=dfData['SS55_CS']+dfData['SS55_FS']+dfData['SS55_FSS']+dfData['SS55_IFS']+dfData['SS55_LS']+dfData['SS55_LSS']
dfData['SS88']=dfData['SS88_CS']+dfData['SS88_FS']+dfData['SS88_FSS']+dfData['SS88_IFS']+dfData['SS88_LS']+dfData['SS88_LSS']

dfData['TempASBCU_SUM']=dfData['SSS1']+dfData['SSS2']+dfData['SSS3']+dfData['SS22']+dfData['SS60']+dfData['SS80']+dfData['SS33']+dfData['SS55']+dfData['SS88']+dfData['SS77']

dfData['SS33_55_88_FS-FSS']=dfData['SS33_FS']+dfData['SS33_FSS']+dfData['SS88_FS']+dfData['SS88_FSS']+dfData['SS55_FS']+dfData['SS55_FSS']
dfData['SS33_55_88_LS-LSS']=dfData['SS33_LS']+dfData['SS33_LSS']+dfData['SS88_LS']+dfData['SS88_LSS']+dfData['SS55_LS']+dfData['SS55_LSS']
dfData['SS33_55_88_CS-IFS']=dfData['SS33_CS']+dfData['SS33_IFS']+dfData['SS88_CS']+dfData['SS88_IFS']+dfData['SS55_CS']+dfData['SS55_IFS']

dfData['SS33_55_88_FS-FSSWeight']=dfData['SS33_55_88_FS-FSS']/dfData['SS33_55_88']
dfData['SS33_55_88_LS-LSSWeight']=dfData['SS33_55_88_LS-LSS']/dfData['SS33_55_88']
dfData['SS33_55_88_CS-IFSWeight']=dfData['SS33_55_88_CS-IFS']/dfData['SS33_55_88']

lstSBY=['SSS1','SS80','SSS2','SSS3','SS22','SS60','SS33','SS55','SS88','SS77']
lstRankCombine=['FS-FSS','LS-LSS','CS-IFS']

for i in lstSBY:
    for j in lstRankCombine:
        var=i+'_'+j
        dfData[var]=dfData[i+'_'+j[0:2]]+dfData[i+'_'+j[3:6]]
        ratio_var=var+'Weight'
        dfData[ratio_var]=round(dfData[var]/dfData[i],2)

dfData['TempFS_FSS']=dfData['SS33_FS']+dfData['SS77_FS']+dfData['SS55_FS']+dfData['SS60_FS']+dfData['SS22_FS']+dfData['SS80_FS']+dfData['SS88_FS']+dfData['SS22_FSS']+dfData['SSS1_FS']+dfData['SSS2_FS']+dfData['SSS3_FS']+dfData['SS33_FSS']+dfData['SS55_FSS']+dfData['SS60_FSS']+dfData['SS80_FSS']+dfData['SS88_FSS']+dfData['SSS1_FSS']+dfData['SSS2_FSS']+dfData['SSS3_FSS']+dfData['SS77_FSS']
    
dfData['TempLS_LSS']=dfData['SS33_LS']+dfData['SS77_LS']+dfData['SS55_LS']+dfData['SS60_LS']+dfData['SS22_LS']+dfData['SS80_LS']+dfData['SS88_LS']+dfData['SS22_LSS']+dfData['SSS1_LS']+dfData['SSS2_LS']+dfData['SSS3_LS']+dfData['SS33_LSS']+dfData['SS55_LSS']+dfData['SS60_LSS']+dfData['SS80_LSS']+dfData['SS88_LSS']+dfData['SSS1_LSS']+dfData['SSS2_LSS']+dfData['SSS3_LSS']+dfData['SS77_LSS']

dfData['TempCS_IFS']=dfData['SS33_CS']+dfData['SS77_CS']+dfData['SS55_CS']+dfData['SS60_CS']+dfData['SS22_CS']+dfData['SS80_CS']+dfData['SS88_CS']+dfData['SS22_IFS']+dfData['SSS1_CS']+dfData['SSS2_CS']+dfData['SSS3_CS']+dfData['SS33_IFS']+dfData['SS55_IFS']+dfData['SS60_IFS']+dfData['SS80_IFS']+dfData['SS88_IFS']+dfData['SSS1_IFS']+dfData['SSS2_IFS']+dfData['SSS3_IFS']+dfData['SS77_IFS']    
    
    
dfData['DayOfWeek']=dfData['DayOfWeek'].astype(np.int64)   
        
        
class ModelParams(object):

    def __init__(self):
        self.ModelName = ""        
        
        self.Features  = []

        self.TargetVar = ""
        self.OutputVar = ""
        self.AllocatedVar = ""
        self.AveragVar = ""

        self.ModelVars = ""

        self.MAE  = 0
        self.RMSE = 0
        self.Current_MAPE=0
        self.MAPE = 0
        self.TS   = 0
        self.SD   = 0
        self.RSq  = 0
        
        self.rgModelCoeff=""
        self.ModelVar=""
        self.NNPred_MAPE=0
        self.LinearRegPred_MAPE=0
        self.GBPred_MAPE=0
        self.PercentileRegPred_MAPE=0
        self.ENetRegPred_MAPE=0
        
        
        self.TrainStartDate = dt.date(2000,1,1)
        self.TrainEndDate   = dt.date(2000,1,1)
        
        self.TestStartDate  = dt.date(2000,1,1)
        self.TestEndDate    = dt.date(2000,1,1)

        self.IFSCSWeights=0
        self.FSFSSWeights=0
        self.LSLSSWeights=0
        
        self.SSS0_Weights=0
        self.SSS1_Weights=0
        self.SSS2_Weights=0
        self.SSS3_Weights=0
        
        self.rgModel=''
        self.GBModel=''
        self.resModel=''
        self.elsModel=''
        self.nnModel=''
        self.GB_params={}
        self.Month = ""
        self.Year = "" 
    

def ModelGraphs(dfTestingData,objMP):
    
    PredictedVar='Total Standby'
    AllocatedVar='PSBCU_SUM'
    dfTestingData[PredictedVar]=dfTestingData[PredictedVar].astype(np.int64)
    count=0
    a=[]
    error=[]
    
    Currentcount=0
    Currenta=[]
    Currenterror=[]

    ## get underforecast instances
    for i in range(0,len(dfTestingData)):
        if (dfTestingData[PredictedVar][i]<dfTestingData[objMP.TargetVar][i]):
            count=count+1
            error.append(dfTestingData[objMP.TargetVar][i] - dfTestingData[PredictedVar][i])
            a.append((dfTestingData[objMP.TargetVar][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    UFMAPE=np.mean(a)*100
    
    avg_error=np.mean(error)
    sd_error=np.std(error)

    ## get underforecast for current process
    
    for i in range(0,len(dfTestingData)):
        if (dfTestingData[AllocatedVar][i]<dfTestingData[objMP.TargetVar][i]):

            Currentcount=Currentcount+1
            Currenterror.append(dfTestingData[objMP.TargetVar][i] - dfTestingData[AllocatedVar][i])
            Currenta.append((dfTestingData[objMP.TargetVar][i] - dfTestingData[AllocatedVar][i]) / dfTestingData[objMP.TargetVar][i])
    print(Currenta)
    CurrentUFMAPE=np.mean(Currenta)*100
    
    
    ## Resourced Saved below 220
    dfResSave=0
    for i in range(0,len(dfTestingData)):
        if ((dfTestingData[PredictedVar][i]>dfTestingData[objMP.TargetVar][i]) & (dfTestingData[PredictedVar][i]<220)):
            dfResource=220-int(dfTestingData[PredictedVar][i])
            dfResSave=dfResSave+dfResource
    
    ## Extra Coverage
    
    dfExtraRequired=0
    for i in range(0,len(dfTestingData)):
        if ((dfTestingData[PredictedVar][i]>dfTestingData[objMP.TargetVar][i]) & (dfTestingData[objMP.TargetVar][i]>220)):
            dfextraresource=int(dfTestingData[objMP.TargetVar][i])-220
            dfExtraRequired=dfExtraRequired+dfextraresource
    
    ## Over forecast
    
    dfExtra=0
    for i in range(0,len(dfTestingData)):
        if ((dfTestingData[PredictedVar][i]>dfTestingData[objMP.TargetVar][i]) & (dfTestingData[PredictedVar][i]>220)):
            dfoverforecast=int(dfTestingData[PredictedVar][i])-int(dfTestingData[objMP.TargetVar][i])
            dfExtra=dfExtra+dfoverforecast

    Overcount=0
    b=[]
    
    ## get overforecast instances
    for i in range(0,len(dfTestingData)):
        if ((dfTestingData[PredictedVar][i]>dfTestingData[objMP.TargetVar][i]) & (dfTestingData[PredictedVar][i]>220)):
            Overcount=Overcount+1
            b.append(abs(dfTestingData[objMP.TargetVar][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    OFMAPE=np.mean(b)*100
    #dfTestingData['Pred_ASBCU_SUM']=dfTestingData['Pred_ASBCU_SUM'].astype(np.int64)
    #print(dfTestingData.dtypes)
    #UnderForecast=dfTestingData[dfTestingData[(PredictedVar<objMP.TargetVar)]]
    #UFMAPE=np.mean(np.abs((UnderForecast[objMP.TargetVar] - UnderForecast[PredictedVar]) / UnderForecast[objMP.TargetVar])) * 100
    
    import matplotlib.pyplot as plt
    fig = plt.figure()
    plt.figure(figsize=(10,5))
    plt.plot(dfTestingData['Day'],dfTestingData[PredictedVar],'b',label='forecasted')
    plt.plot(dfTestingData['Day'],dfTestingData[objMP.TargetVar],'r',label='actual')
    plt.plot(dfTestingData['Day'],dfTestingData[AllocatedVar],'g',label='allocated')
    plt.axhline(260, color="orange",label='Standard 260')
    plt.ylim(100,400)
    plt.xlim(1,31)
    #plt.axhline(y=.4, xmin=0.25, xmax=0.402, linewidth=2, color = 'k')
    plt.xlabel('Day')
    plt.ylabel('Crew Count')
    #plt.title(objMP.TargetVar[6:]+' Rank Forecast '+objMP.Month+' '+objMP.Year)
    plt.title('Agg. Standby Forecast'+' '+objMP.Month+' '+objMP.Year)
    
    plt.legend(loc='upper right',fancybox=True, framealpha=0.5)
    #ax.legend(loc='best', fancybox=True, framealpha=0.5)
    #Remove below comments
    #plt.text(0.15, 0.85, objMP.ModelName,fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.80, "Current Process:  MAPE: "+str(round(objMP.Current_MAPE, 2))+", "+"UF: "+str(round(Currentcount,2))+", "+"MAPE(UF): "+str(round(CurrentUFMAPE,2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.80, "Current Process:  MAPE: "+str(round(objMP.Current_MAPE, 2)), fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.75, "Model MAPE: "+str(round(objMP.MAPE, 2))+", "+"UF: "+str(round(count, 2))+", "+"MAPE(UF): "+str(round(UFMAPE, 2))+", "+"MAPE Ratio: "+str(round((objMP.MAPE/objMP.Current_MAPE), 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.75, "MAE: "+str(round(objMP.MAE, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.70, "MAPE: "+str(round(objMP.MAPE, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.65, "Tracking Signal: "+str(round(objMP.TS, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.75, "# UF: "+str(round(count, 2))+", "+"MAPE(UF): "+str(round(UFMAPE, 2)), fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.70, "Avg_Error(UF): "+str(round(avg_error, 2))+", "+"SD_Error(UF): "+str(round(sd_error, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.70, "%age UF: "+str(round(UFMAPE, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.70, "Crew Saved: "+str(round(dfResSave, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.60, "Extra Coverage : "+str(round(dfExtraRequired, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.70, "# OF : "+str(round(Overcount, 2))+","+"OF %: "+str(round(OFMAPE, 2)), fontsize=10, transform=plt.gcf().transFigure)
    #plt.savefig('D:/Proof of Concept/Crew Planning Analytics/Client Meeting/June 29/Updated/'+objMP.Month+' '+objMP.Year+'.png')
    #plt.savefig('D:/Proof of Concept/Crew Planning Analytics/Client Meeting/July 23/final/graphs/Agg. Standby Forecast'+objMP.Month+'.png')
    plt.show()
    
    return



def RankModelGraph(dfTestingData,objMP):
    
    import matplotlib.pyplot as plt
    
    modelfscount=0
    fsa=[]
    modelfserror=[]
    Currentfscount=0
    Currentfserror=[]
    
    ## get underforecast instances
    for i in range(0,len(dfTestingData)):
        if (dfTestingData['Total FS-FSS'][i]<dfTestingData['TempFS_FSS'][i]):
            modelfscount=modelfscount+1
            modelfserror.append( dfTestingData['TempFS_FSS'][i]-dfTestingData['Total FS-FSS'][i])
            #fsa.append((dfTestingData['Total FS-FSS'][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    modelfsavg_error=np.mean(modelfserror)
    modelfssd_error=np.std(modelfserror)
     
    for i in range(0,len(dfTestingData)):
        if (dfTestingData['PSBCU_FS-FSS'][i]<dfTestingData['TempFS_FSS'][i]):
            Currentfscount=Currentfscount+1
            Currentfserror.append( dfTestingData['TempFS_FSS'][i]-dfTestingData['PSBCU_FS-FSS'][i])
            #Currentfsa.append((dfTestingData['Total FS-FSS'][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    Currentfsavg_error=np.mean(Currentfserror)
    Currentfssd_error=np.std(Currentfserror) 
        
        
    fig = plt.figure()
    plt.figure(figsize=(10,5))
    #dfTestingData['Pred_ASBCU_IFS-CS']=objMP.IFSCSWeights*dfTestingData['Pred_ASBCU_SUM'] 
    #dfTestingData['Pred_ASBCU_IFS-CS']=round(dfTestingData['Pred_ASBCU_IFS-CS'],0)
    plt.plot(dfTestingData['Day'],dfTestingData['Total FS-FSS'],'b',label='forecasted')
    
    plt.plot(dfTestingData['Day'],(dfTestingData['TempFS_FSS']),'r',label='actual')
    plt.plot(dfTestingData['Day'],dfTestingData['PSBCU_FS-FSS'],'g',label='allocated')
    
    plt.xlim(1,31)
    plt.ylim(50,250)
    #axes[0,1].set_xlabel('Day')
    plt.ylabel('Crew Count')
    plt.title(' FS-FSS Rank Forecast '+objMP.Month+' '+objMP.Year)
    plt.legend(loc='upper right',fancybox=True, framealpha=0.5)
    #objMP.MAPE=np.mean(np.abs(((dfTestingData['ASBCU_IFS']+dfTestingData['ASBCU_CS']) - dfTestingData['Pred_ASBCU_IFS-CS']) / (dfTestingData['ASBCU_IFS']+dfTestingData['ASBCU_CS']))) * 100
    #axes[0,1].legend(loc='best', fancybox=True, framealpha=0.5)
    #plt.text(0.15, 0.85, "Current Process: # UF: "+str(round(Currentfscount, 2)),fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.85, 'Current Process: #UF: '+str(round(Currentfscount, 2))+', '+'Avg_Error: '+str(round(Currentfsavg_error, 2))+", "+"SD_Error(UF): "+str(round(Currentfssd_error, 2)),fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.80, "Model: # UF: "+str(round(modelfscount, 2))+', '+'Avg_Error: '+str(round(modelfsavg_error, 2))+", "+"SD_Error(UF): "+str(round(modelfssd_error, 2)),fontsize=10, transform=plt.gcf().transFigure)
    #plt.savefig('D:/Proof of Concept/Crew Planning Analytics/Client Meeting/July 23/final/graphs/FS-FSS Rank Forecast'+objMP.Month+'.png')
    plt.show()
    
    plt.figure(figsize=(10,5))
    
    
    modelifscount=0
    Currentifsa=[]
    modelifsa=[]
    Currentifserror=[]
    modelifserror=[]
    Currentifscount=0
    ## get underforecast instances for model 
    for i in range(0,len(dfTestingData)):
        if (dfTestingData['Total CS-IFS'][i]<dfTestingData['TempCS_IFS'][i]):
            modelifscount=modelifscount+1
            modelifserror.append(dfTestingData['TempCS_IFS'][i]-dfTestingData['Total CS-IFS'][i])
            #modelifsa.append((dfTestingData['Total FS-FSS'][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    modelifsavg_error=np.mean(modelifserror)
    modelifssd_error=np.std(modelifserror)
    
    for i in range(0,len(dfTestingData)):
        if (dfTestingData['PSBCU_IFS-CS'][i]<dfTestingData['TempCS_IFS'][i]):
            Currentifscount=Currentifscount+1
            Currentifserror.append(dfTestingData['TempCS_IFS'][i]-dfTestingData['PSBCU_IFS-CS'][i])
            #Currentfsa.append((dfTestingData['Total FS-FSS'][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    Currentifsavg_error=np.mean(Currentifserror)
    Currentifssd_error=np.std(Currentifserror)    
    
    
    
    
    plt.plot(dfTestingData['Day'],dfTestingData['Total CS-IFS'],'b',label='forecasted')
    plt.plot(dfTestingData['Day'],(dfTestingData['TempCS_IFS']),'r',label='actual')
    plt.plot(dfTestingData['Day'],dfTestingData['PSBCU_IFS-CS'],'g',label='allocated')
    plt.xlim(1,31)
    plt.ylim(0,100)
    #axes[0,1].set_xlabel('Day')
    plt.ylabel('Crew Count')
    plt.title(' CS-IFS Rank Forecast '+objMP.Month+' '+objMP.Year)
    plt.legend(loc='upper right',fancybox=True, framealpha=0.5)
    #objMP.MAPE=np.mean(np.abs(((dfTestingData['ASBCU_FS']+dfTestingData['ASBCU_FSS']) - dfTestingData['Pred_ASBCU_FS-FSS']) / (dfTestingData['ASBCU_FS']+dfTestingData['ASBCU_FSS']))) * 100
    #axes[0,1].legend(loc='best', fancybox=True, framealpha=0.5)
    #plt.text(0.15, 0.85, objMP.ModelName,fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.85, 'Current Process: #UF: '+str(round(Currentifscount, 2))+', '+'Avg_Error: '+str(round(Currentifsavg_error, 2))+", "+"SD_Error(UF): "+str(round(Currentifssd_error, 2)),fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.80, "Model: # UF: "+str(round(modelifscount, 2))+', '+'Avg_Error: '+str(round(modelifsavg_error, 2))+", "+"SD_Error(UF): "+str(round(modelifssd_error, 2)),fontsize=10, transform=plt.gcf().transFigure)
    #plt.savefig('D:/Proof of Concept/Crew Planning Analytics/Client Meeting/July 23/final/graphs/CS-IFS Rank Forecast'+objMP.Month+'.png')
    plt.show()
    
    plt.figure(figsize=(10,5))
    
    
    modellscount=0
    la=[]
    modellserror=[]
    Currentlscount=0
    Currentlserror=[]
    ## get underforecast instances
    for i in range(0,len(dfTestingData)):
        if (dfTestingData['Total LS-LSS'][i]<dfTestingData['TempLS_LSS'][i]):
            modellscount=modellscount+1
            modellserror.append(dfTestingData['TempLS_LSS'][i]-dfTestingData['Total LS-LSS'][i])
            #fsa.append((dfTestingData['Total FS-FSS'][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    modellsavg_error=np.mean(modellserror)
    modellssd_error=np.std(modellserror)
    
    for i in range(0,len(dfTestingData)):
        if (dfTestingData['PSBCU_LS-LSS'][i]<dfTestingData['TempLS_LSS'][i]):
            Currentlscount=Currentlscount+1
            Currentlserror.append(dfTestingData['TempLS_LSS'][i]-dfTestingData['PSBCU_LS-LSS'][i])
            #fsa.append((dfTestingData['Total FS-FSS'][i] - dfTestingData[PredictedVar][i]) / dfTestingData[objMP.TargetVar][i])
    Currentlsavg_error=np.mean(Currentlserror)
    Currentlssd_error=np.std(Currentlserror)    
    
    
    
    
    plt.plot(dfTestingData['Day'],dfTestingData['Total LS-LSS'],'b',label='forecasted')
    plt.plot(dfTestingData['Day'],dfTestingData['TempLS_LSS'],'r',label='actual')
    plt.plot(dfTestingData['Day'],dfTestingData['PSBCU_LS-LSS'],'g',label='allocated')
    plt.xlim(1,31)
    plt.ylim(0,200)
    #axes[0,1].set_xlabel('Day')
    plt.ylabel('Crew Count')
    plt.title(' LS-LSS Rank Forecast '+objMP.Month+' '+objMP.Year)
    plt.legend(loc='upper right',fancybox=True, framealpha=0.5)
    #objMP.MAPE=np.mean(np.abs(((dfTestingData['ASBCU_LS']+dfTestingData['ASBCU_LSS']) - dfTestingData['Pred_ASBCU_LS-LSS']) / (dfTestingData['ASBCU_LS']+dfTestingData['ASBCU_LSS']))) * 100
    #axes[0,1].legend(loc='best', fancybox=True, framealpha=0.5)
    #plt.text(0.15, 0.85, "Current Process: # UF: "+str(round(Currentlscount, 2)),fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.85, 'Current Process: #UF: '+str(round(Currentlscount, 2))+', '+'Avg_Error: '+str(round(Currentlsavg_error, 2))+", "+"SD_Error(UF): "+str(round(Currentlssd_error, 2)),fontsize=10, transform=plt.gcf().transFigure)
    #plt.text(0.15, 0.80, 'MAPE: '+str(round(objMP.MAPE, 2)),fontsize=10, transform=plt.gcf().transFigure)
    plt.text(0.15, 0.80, "Model: # UF: "+str(round(modellscount, 2))+', '+'Avg_Error: '+str(round(modellsavg_error, 2))+", "+"SD_Error(UF): "+str(round(modellssd_error, 2)),fontsize=10, transform=plt.gcf().transFigure)
    #plt.savefig('D:/Proof of Concept/Crew Planning Analytics/Client Meeting/July 23/final/graphs/LS-LSS Rank Forecast'+objMP.Month+'.png')
    plt.show()
    
    return(dfTestingData)


def ConverttoCategory(data,collist):
    for column in collist:
        data[column]=data[column].astype('category')
    return(data)

def OneHotEncoding(data,collist):
    for column in collist:
        data=pd.get_dummies(data, columns=[column],drop_first=True)
        
    return(data)

def ModelEvaluation(dfTestingData,objMP):    
    
    PredictedVar='Total Standby'
    AllocatedVar='PSBCU_SUM'
    objMP.Current_MAPE=np.mean(np.abs((dfTestingData[objMP.TargetVar] - dfTestingData[AllocatedVar]) / dfTestingData[objMP.TargetVar])) * 100
    
    objMP.MAE = sm.mean_absolute_error(dfTestingData[objMP.TargetVar], dfTestingData[PredictedVar])
    objMP.RMSE = math.sqrt(sm.mean_squared_error(dfTestingData[objMP.TargetVar], dfTestingData[PredictedVar]))
    objMP.MAPE=np.mean(np.abs((dfTestingData[objMP.TargetVar] - dfTestingData[PredictedVar]) / dfTestingData[objMP.TargetVar])) * 100
    objMP.TS=sum(dfTestingData[PredictedVar]-dfTestingData[objMP.TargetVar]) / objMP.MAE
    
    return


def RegressionAlgorithm(dfTrainingData,Predictors,objMP):
    
    
    objMP.ModelName='Regression'

    lm = linear_model.LinearRegression()
    objMP.rgModel=lm.fit(dfTrainingData[Predictors],dfTrainingData[objMP.TargetVar])

    return


def GradientBoost(dfTrainingData,Predictors,objMP):
    


    objMP.ModelName='GradientBoost'


    clf = ensemble.GradientBoostingRegressor(**objMP.GB_params)

    objMP.GBModel=clf.fit(dfTrainingData[Predictors],dfTrainingData[objMP.TargetVar])

    return

def ElasticNetAlgorithm(data,Predictors,objMP):
    

    objMP.ModelName='ElasticNet'
    
    els = ElasticNet(random_state=0, l1_ratio=objMP.dictparam['L1Ratio'])
    
    objMP.elsModel=els.fit(data[Predictors],data[objMP.TargetVar])

    return 

def QuantileRegressionAlgorithm(dfTrainingData,Predictors,objMP):
    

    
    objMP.ModelName='QuantileRegression'

    objMP.ModelVar=objMP.TargetVar + '~' +  '+'.join(Predictors)
    print(objMP.ModelVar)

    mod = smf.quantreg(objMP.ModelVar,dfTrainingData)
    objMP.resModel = mod.fit(objMP.dictparam['quantile'])
    
    return

def NeuralNetwork(data,lstPredictor,objMP):
    
    objMP.ModelName='NN'
    mlp = nn.MLPRegressor(hidden_layer_sizes=(3),batch_size = 'auto',max_iter = 10000, verbose = True, shuffle = True, random_state = 1)
    
    objMP.nnModel = mlp.fit(data[lstPredictor],data[objMP.TargetVar])
    return

def SetVariables(item):
    
    Pred=[]
    objMP.TargetVar=item
    
    if ((item=='SS33') | (item=='SS55') | (item=='SS88')):
        Pred=['PCOP_SS33_55_88_Sum']
    else:
        Ranks=['Sum']
        Pred=['PCOP_'+item+'_'+i for i in Ranks]
    return(Pred,objMP)

def SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes):
    
    objMP.dictparam={'quantile':q,
               'L1Ratio':L1Ratio,
               'hidden_layer_sizes':hidden_layer_sizes
              }
    
    objMP.GB_params=GB_params
    
    return(objMP)

def FitModels(dfData,Predlist,objMP):
    
    RegressionAlgorithm(dfTrainingData,Predlist,objMP)
    GradientBoost(dfTrainingData,Predlist,objMP)
    ElasticNetAlgorithm(dfTrainingData,Predlist,objMP)
    NeuralNetwork(dfTrainingData,Predlist,objMP)
    QuantileRegressionAlgorithm(dfTrainingData,Predlist,objMP)
    return(objMP)

def PredictModel(dfTestingData,Predlist,objMP):
    
    dfTestingData.insert(0, 'LinearRegPred_'+objMP.TargetVar, (objMP.rgModel.predict(dfTestingData[Predlist])))
    dfTestingData.insert(0, 'GBPred_'+objMP.TargetVar,( objMP.GBModel.predict(dfTestingData[Predlist])))
    dfTestingData.insert(0, 'NNPred_'+objMP.TargetVar,( objMP.nnModel.predict(dfTestingData[Predlist])))
    dfTestingData.insert(0, 'PercentileRegPred_'+objMP.TargetVar, (objMP.resModel.predict(dfTestingData[Predlist])))
    dfTestingData.insert(0, 'ENetRegPred_'+objMP.TargetVar, (objMP.elsModel.predict(dfTestingData[Predlist])))
    return(dfTestingData)

def RankModel(dfTrainingData,dfTestingData,item,strOutput):

    fs_fss=dfTrainingData[item+'_FS-FSSWeight'].tail(30).mean()
    ls_lss=dfTrainingData[item+'_LS-LSSWeight'].tail(30).mean()
    cs_ifs=dfTrainingData[item+'_CS-IFSWeight'].tail(30).mean()
    
    dfTestingData['Pred_'+item+'_FS-FSS']=round(dfTestingData[strOutput]*fs_fss,0)
    dfTestingData['Pred_'+item+'_LS-LSS']=round(dfTestingData[strOutput]*ls_lss,0)
    dfTestingData['Pred_'+item+'_CS-IFS']=round(dfTestingData[strOutput]*cs_ifs,0)
    
    return(dfTestingData)

Predictor=['Month','Year','WeekofMonth','isHoliday','DayNum']
lstSScodes=['SSS1','SS80','SSS2','SSS3','SS22','SS60','SS33','SS55','SS88','SS77']  


Category_variable=list(['Month','WeekofMonth','isHoliday','DayOfWeek'])
Encode_variable=list(['DayOfWeek'])

#dfData['Date']=pd.to_datetime(dfData['Date'],format="%Y-%m-%d")
dfData['Date']=pd.to_datetime(dfData['Date'],format='%m/%d/%Y')
dfData['DayWeek']=dfData['DayOfWeek'].copy()
dfData['Vacation_flag']=np.where((dfData['Date'].dt.month==6) | (dfData['Date'].dt.month==12),1,0)
dfData['MonthNum'] = dfData['Month'].ne(dfData['Month'].shift().bfill()).astype(int)
dfData['MonthNum'].iloc[0]=1
dfData['MonthNum']=dfData['MonthNum'].cumsum().astype(np.int64)
dfData['DayNum']=range(1, 1 + len(dfData))
dfData['DayNum']=dfData['DayNum'].astype(np.int64)
if Category_variable:
    dfData=ConverttoCategory(dfData,Category_variable)
    
if  Encode_variable:
    orgColNames =  list(dfData.columns.values)
    dfData=OneHotEncoding(dfData,Encode_variable)
    newColNames = list(dfData.columns.values)
    Predictor_var = list(set(newColNames).difference(orgColNames))
    dfData=ConverttoCategory(dfData,Predictor_var)
        
else:
    Predictor_var=[]
    
Predictor.extend(Predictor_var)

############ Model Input ####################


objMP = ModelParams()
objMP.TrainStartDate = dt.date(2016,3,1)
objMP.TrainEndDate = dt.date(2018,5,31)

objMP.TestStartDate = dt.date(2018,6,1)
objMP.TestEndDate = dt.date(2018,6,30) 
    
objMP.Month = objMP.TestStartDate.strftime("%B")
objMP.Year = objMP.TestStartDate.strftime("%Y")

############ Model Parameters ################
#gradient boost
GB_params = {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 5,
       'learning_rate': 0.015, 'loss': 'ls','random_state':10}
# percentile
q=0.6

# elasticnet
L1Ratio=0.15
#NN
hidden_layer_sizes=5

##############################################
dfTrainingData=dfData[(dfData['Date'] >= objMP.TrainStartDate) & (dfData['Date'] <= objMP.TrainEndDate)].reset_index(drop = True)

dfTestingData=dfData[(dfData['Date'] >= objMP.TestStartDate) & (dfData['Date'] <= objMP.TestEndDate)].reset_index(drop = True)    
    
for item in lstSScodes:
    
    Predlist,objMP=SetVariables(item)
    Predlist.extend(Predictor)
    print(Predlist)
    
    if item == 'SSS1':
        q=0.9
        L1Ratio=0.15
        GB_params = {'n_estimators': 200, 'max_depth': 10, 'min_samples_split': 5,
       'learning_rate': 0.015, 'loss': 'ls','random_state':10}
        
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        dfTestingData['Ensemble'+objMP.TargetVar]=(dfTestingData['LinearRegPred_'+objMP.TargetVar]+dfTestingData['NNPred_'+objMP.TargetVar]+dfTestingData['PercentileRegPred_'+objMP.TargetVar])/3
        strOutput='EnsembleSSS1'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
        
    elif item=='SSS2':
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='NNPred_SSS2'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
    
    elif item=='SSS3':
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='LinearRegPred_SSS3'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
        
    elif item=='SS22':
        q=0.9
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='PercentileRegPred_SS22'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
    elif item=='SS60':
        q=0.9
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='PercentileRegPred_SS60'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
        
    elif item=='SS80':
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='NNPred_SS80'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
        
    elif item=='SS33':
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='PercentileRegPred_SS33'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
    elif item=='SS55':
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='PercentileRegPred_SS55'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
    elif item=='SS88':
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)
        strOutput='PercentileRegPred_SS88'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
    else:
        objMP=SetModelParams(q,GB_params,L1Ratio,hidden_layer_sizes)
        objMP=FitModels(dfTrainingData,Predlist,objMP)
        dfTestingData=PredictModel(dfTestingData,Predlist,objMP)    
        dfTestingData['Ensemble'+objMP.TargetVar]=(dfTestingData['NNPred_'+objMP.TargetVar]+dfTestingData['GBPred_'+objMP.TargetVar])/2
        strOutput='EnsembleSS77'
        dfTestingData[strOutput]=round(dfTestingData[strOutput],0)
        dfTestingData=RankModel(dfTrainingData,dfTestingData,item,strOutput)
        
## Operational Exigencies

dfTestingData['Pred_AOG']=23    
dfTestingData['Pred_AOG_CS-IFS']=8
dfTestingData['Pred_AOG_FS-FSS']=8
dfTestingData['Pred_AOG_LS-LSS']=7

objMP.TargetVar='TempASBCU_SUM'
    

dfTestingData['Total FS-FSS']=dfTestingData[['Pred_SSS1_FS-FSS','Pred_SSS2_FS-FSS','Pred_SSS3_FS-FSS','Pred_SS22_FS-FSS',
                                            'Pred_SS60_FS-FSS','Pred_SS80_FS-FSS','Pred_SS33_FS-FSS','Pred_SS55_FS-FSS',
                                            'Pred_SS88_FS-FSS','Pred_SS77_FS-FSS']].sum(axis=1)+dfTestingData['Pred_AOG_FS-FSS']

dfTestingData['Total LS-LSS']=dfTestingData[['Pred_SSS1_LS-LSS','Pred_SSS2_LS-LSS','Pred_SSS3_LS-LSS','Pred_SS22_LS-LSS',
                                            'Pred_SS60_LS-LSS','Pred_SS80_LS-LSS','Pred_SS33_LS-LSS','Pred_SS55_LS-LSS',
                                            'Pred_SS88_LS-LSS','Pred_SS77_LS-LSS']].sum(axis=1)+dfTestingData['Pred_AOG_LS-LSS']

dfTestingData['Total CS-IFS']=dfTestingData[['Pred_SSS1_CS-IFS','Pred_SSS2_CS-IFS','Pred_SSS3_CS-IFS','Pred_SS22_CS-IFS',
                                            'Pred_SS60_CS-IFS','Pred_SS80_CS-IFS','Pred_SS33_CS-IFS','Pred_SS55_CS-IFS',
                                            'Pred_SS88_CS-IFS','Pred_SS77_CS-IFS']].sum(axis=1)+dfTestingData['Pred_AOG_CS-IFS']

dfTestingData['Total Standby']=dfTestingData[['Total FS-FSS','Total LS-LSS','Total CS-IFS']].sum(axis=1)

ModelEvaluation(dfTestingData,objMP)
ModelGraphs(dfTestingData,objMP)
RankModelGraph(dfTestingData,objMP)


lstOutput=['Pred_AOG_CS-IFS','Pred_AOG_FS-FSS','Pred_AOG_LS-LSS','Pred_SSS1_FS-FSS',
       'Pred_SSS1_LS-LSS', 'Pred_SSS1_CS-IFS', 'Pred_SS80_FS-FSS',
       'Pred_SS80_LS-LSS', 'Pred_SS80_CS-IFS', 'Pred_SSS2_FS-FSS',
       'Pred_SSS2_LS-LSS', 'Pred_SSS2_CS-IFS', 'Pred_SSS3_FS-FSS',
       'Pred_SSS3_LS-LSS', 'Pred_SSS3_CS-IFS', 'Pred_SS22_FS-FSS',
       'Pred_SS22_LS-LSS', 'Pred_SS22_CS-IFS', 'Pred_SS60_FS-FSS',
       'Pred_SS60_LS-LSS', 'Pred_SS60_CS-IFS', 'Pred_SS33_FS-FSS',
       'Pred_SS33_LS-LSS', 'Pred_SS33_CS-IFS', 'Pred_SS55_FS-FSS',
       'Pred_SS55_LS-LSS', 'Pred_SS55_CS-IFS', 'Pred_SS88_FS-FSS',
       'Pred_SS88_LS-LSS', 'Pred_SS88_CS-IFS', 'Pred_SS77_FS-FSS',
       'Pred_SS77_LS-LSS', 'Pred_SS77_CS-IFS']


dfOutput=dfTestingData[lstOutput].copy()

lstOutput=[i.strip('Pred_') for i in lstOutput]
dfOutput.columns=lstOutput

lstOutputvar=['Date','Total Standby','Total FS-FSS','Total LS-LSS','Total CS-IFS']
dfOutput[lstOutputvar]=dfTestingData[lstOutputvar].copy()

dfFinal=dfOutput[lstOutputvar].copy()
dfFinal[lstOutput]=dfOutput[lstOutput].copy()


